In [ ]:
import logging
import time
import os
import pickle
import math

import matplotlib.pyplot as plt

#import tensorflow_datasets as tfds
import tensorflow as tf

# Import tf_text to load the ops used by the tokenizer saved model
#import tensorflow_text  # pylint: disable=unused-import
import pandas as pd
import numpy as np
import re
import seaborn as sns

from sklearn.model_selection import train_test_split

import jellyfish 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model,  Sequential
from tensorflow.keras.layers import LSTM, GRU, Bidirectional, Dropout, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding, Concatenate
import tensorflow.keras.layers as layers
from tensorflow.keras.layers import Attention
from tensorflow.keras.optimizers import Adam, Adagrad
from keras.losses import sparse_categorical_crossentropy
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings
import random

In [ ]:
file_names = ['BosT','Human','Pichia',   'Mouse']

In [ ]:
def AA_Codon_list():
    dic_AA_codon = {'A': ['GCT', 'GCC', 'GCA', 'GCG'],
                    'C': ['TGT', 'TGC'],
                    'D': ['GAT', 'GAC'],
                    'E': ['GAA', 'GAG'],
                    'F': ['TTT', 'TTC'],
                    'G': ['GGT', 'GGA', 'GGC', 'GGG'],
                    'H': ['CAT', 'CAC'],
                    'I': ['ATT', 'ATC', 'ATA'],
                    'K': ['AAA', 'AAG'],
                    'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
                    'M': ['ATG'],
                    'N': ['AAT', 'AAC'],
                    'P': ['CCT', 'CCC', 'CCA', 'CCG'],
                    'Q': ['CAA', 'CAG'],
                    'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
                    'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
                    'T': ['ACT', 'ACC', 'ACA', 'ACG'],
                    'V': ['GTT', 'GTC', 'GTA', 'GTG'],
                    'W': ['TGG'],
                    'Y': ['TAT', 'TAC'],
                    '*': ['TAA', 'TAG', 'TGA']}

    codon_list = []
    AA_list = []
    for key in dic_AA_codon:
        for i in range(len(dic_AA_codon[key])):
            AA_list.append(key)
        for i in dic_AA_codon[key]:
            codon_list.append(i)
    return AA_list, codon_list

## GC content

## CSI

## %Rare codon

In [ ]:
Data = {}
Prop = {}
Codon_Table = {}
Max_freq = {}

for i in range(1):
    Codon_Table[file_names[i]] = pd.read_csv("./CT/"+ file_names[i] + ".csv")
    Max_freq[file_names[i]] = np.zeros((Codon_Table[file_names[i]].shape[0],1))
    
    for c in range(Codon_Table[file_names[i]].shape[0]):
        subset = Codon_Table[file_names[i]].loc[Codon_Table[file_names[i]].iloc[:, 0]==Codon_Table[file_names[i]].iloc[c, 0], :]
        
        Max_freq[file_names[i]][c,0:1] = subset['relative_frequency'].max()
    
    Codon_Table[file_names[i]] = pd.concat((Codon_Table[file_names[i]], pd.DataFrame(Max_freq[file_names[i]][:,0:1])), axis = 1)
    Codon_Table[file_names[i]].columns = ['AA', 'codon', 'relative_frequency', 'Max_frequency']

In [ ]:
Data_dict_New = {}
for i in range(1): #len(file_names)
    Data[file_names[i]] = pd.read_csv("./Genomes/"+ file_names[i] + ".csv")
    index = []
    for j in range(len( Data[file_names[i]]['CDS_Seq'])):
        if Data[file_names[i]]['CDS_Seq'][j].find('N') == -1:
            index.append(j)

    Data_dict_New[file_names[i]] = Data[file_names[i]].iloc[index, :]
    pd.DataFrame(Data_dict_New[file_names[i]]).to_csv("'BosT_N_removed.csv")
        
    Prop[file_names[i]] = np.zeros((Data_dict_New[file_names[i]].shape[0], 3))
    for l in range(Data_dict_New[file_names[i]].shape[0]):
        seq = Data_dict_New[file_names[i]]['CDS_Seq'].iloc[l]
        seq = seq.replace('T','U')
        Prop[file_names[i]][l,0] = (seq.count('G') + seq.count('C'))/len(seq) # GC content
        for k in np.arange(0,len(seq),3):
            temp = Codon_Table[file_names[i]].loc[Codon_Table[file_names[i]].iloc[:, 1]==seq[k:k+3], :]
            if temp['relative_frequency'].values <0.3:
                Prop[file_names[i]][l,1] = Prop[file_names[i]][l,1] + 1 #rare codons 
            
            Prop[file_names[i]][l,2] = Prop[file_names[i]][l,2] + np.log(temp['relative_frequency'].values/temp['Max_frequency'].values)
            
        Prop[file_names[i]][l,1] = Prop[file_names[i]][l,1]/(len(seq)/3)
        Prop[file_names[i]][l,2] = np.exp(Prop[file_names[i]][l,2]/(len(seq)/3))
        
    df = pd.DataFrame(Prop[file_names[i]])
    df.columns = ['GC','Rare','CSI']
    df.to_excel("'BosT_GenomeScore.xlsx")

In [ ]:
df = pd.DataFrame(Prop[file_names[i]])
df.columns = ['GC','Rare','CSI']
df.to_excel("Pichia_GenomeScore.xlsx")

In [ ]:
plt.boxplot(Prop[file_names[i]][:,0])

In [ ]:
plt.boxplot(Prop[file_names[i]][:,1])

In [ ]:
plt.boxplot(Prop[file_names[i]][:,2])

In [ ]:
 np.log(temp['relative_frequency'].values/temp['Max_frequency'].values)

In [ ]:
temp['relative_frequency'].values

In [ ]:
temp

In [ ]:
seq[k:k+3]